In [ ]:
import param
import panel as pn
import plotly.graph_objects as go
pn.extension('plotly')
import plotly.graph_objs as go
import plotly.express as px

bar_count = 0
pie_count = 0
Overall_Lines = []


class Chart_view(param.Parameterized):

    global Overall_Lines
    global bar_count
    global pie_count

    String = param.String()
    SubjectLines = []
    action = param.Action(lambda x: x.param.trigger('action'), label='Click here!')
    
    @param.depends('action')
    def get_number(self):
        global Overall_Lines
        global bar_count
        global pie_count
        if bar_count == 0:
            labels = ['enthusiasm', 'urgency', 'surprise', 'trust', 'curiosity', 'exclusivity']
            values = [0.1868483 , 0.21882287, 0.09829938, 0.21602127, 0.10822593, 0.17178226]
            d_fig = go.Figure()
            d_fig.add_trace(go.Bar(
                x= labels,
                y= values,
                name= "How do you like our new product",
                marker_color= '#EE94E5'
            ))
            d_fig.update_xaxes(title_text="Emotions")
            d_fig.update_yaxes(title_text="Intensity")
            d_fig.update_xaxes(showgrid=False)
            d_fig.update_yaxes(showgrid=False)
            d_fig.update_layout(title='Subject Line Comparison', title_x=0.5, width=800, height=400, margin=dict(t=100))
            d_fig.update_layout(plot_bgcolor= 'rgba(0,0,0,0)',paper_bgcolor= 'rgba(0,0,0,0)')
            bar_count +=1
            
            return pn.pane.Plotly(d_fig, config = dict(
                    displayModeBar=False
                ) )  
            
        if bar_count == 6:
            return pn.pane.Plotly(fig)
        
        if (self.String=='' or self.String=='reset'):
            fig = go.Figure()
            return pn.pane.Plotly(fig)
        
        self.SubjectLines += [self.String]
        Overall_Lines.append(self.String)
        
        labels = ['enthusiasm', 'urgency', 'surprise', 'trust', 'curiosity', 'exclusivity']
        
        Emotions = ['enthusiasm', 'urgency', 'surprise', 'trust', 'curiosity', 'exclusivity']
        email_colors = ['#EE94E5', '#9DE7B2', '#64ACEF', '#94DFF6', '#FDFBC2' ]

        
        fig1 = go.Figure()
        
        
        for i in range(len(self.SubjectLines)):
            fig1.add_trace(go.Bar(
                x= Emotions,
                y= emotion_predict(self.SubjectLines[i]),
                name= ' '.join(self.SubjectLines[i].split(' ')[0:5]),
                marker_color= email_colors[i]
            ))

       
        fig1.update_xaxes(title_text="Emotions")
        fig1.update_yaxes(title_text="Intensity")
        fig1.update_layout(title='Subject Line Comparison',title_x=0.5, width=800, height=400)
        fig1.update_layout(plot_bgcolor= 'rgba(0,0,0,0)',paper_bgcolor= 'rgba(0,0,0,0)')
        fig1.update_xaxes(showgrid=False)
        fig1.update_yaxes(showgrid=False)
        
        
        bar_count+=1
        
        return pn.pane.Plotly(fig1, config=dict(
                    displayModeBar=False
                ))
    
    @param.depends('action')
    def pie_view(self):
        global Overall_Lines
        global pie_count
        
        labels = ['enthusiasm', 'urgency', 'surprise', 'trust', 'curiosity', 'exclusivity']
        emotion_colors = ['#E7D69D', '#904970', '#5166B1', '#E0A347', '#587D9E', '#A4933C']
        
        if pie_count == 0:
            default_fig = go.Figure(data=[go.Pie(
            labels=labels, 
            values=[0.1868483 , 0.21882287, 0.09829938, 0.21602127, 0.10822593, 0.17178226], 
            textinfo='label+percent', 
            hole=.5)])
            
            default_fig.update_layout(title_text="Subject: How do you like our new product", title_x=0.5)
            default_fig.update_traces(marker=dict(colors=emotion_colors))
            default_fig.update_layout(showlegend=False)
            pie_count += 1
            
            return pn.pane.Plotly(default_fig, config=dict(
                    displayModeBar=False
                )) 
            
        
        #create a title indicating which subject line for pie chart 
        pie_title = 'Subject Line: ' + self.String
        
        fig = go.Figure(data=[go.Pie(
            labels=labels, 
            values=emotion_predict(self.String), 
            textinfo='label+percent', 
            hole=.5)])
        
        
        fig.update_layout(title_text=pie_title, title_x=0.5)
        fig.update_traces(marker=dict(colors=emotion_colors))
        fig.update_layout(showlegend=False)
        
        return pn.pane.Plotly(fig, config=dict(displayModeBar=False))        
    
    # table of data for the chosen subject line
    def table_view(self):
        data = self.get_data()
        return data

charts = Chart_view() 
pn.Column(
    pn.Row(pn.panel(charts.param, show_labels=False, show_name=False, margin=(0, 480, 0, 550), align='end')),
    pn.Row(pn.panel('Please type \"reset\" if you\'d like to refresh the graph', margin=(0, 480, 0, 555))),
    pn.Row(charts.pie_view, charts.get_number)
).servable()   